# 因子分析 -- out_sample 

In [10]:
import pandas as pd
import numpy as np
data = pd.read_parquet("/home/cheam/IP_alpha/data/factor_all.parquet")[['AdjVWAP']]
data.index.get_level_values('Date').unique()
data.reset_index(inplace=True)
data['Date'] = pd.to_datetime(data['Date'])
data.set_index(['Date', 'Symbol'], inplace=True)
data = data.groupby('Symbol').resample('W-MON', level='Date').last()
data.reset_index(inplace=True)
data.set_index(['Date', 'Symbol'], inplace=True)
data.sort_values(['Date', 'Symbol'], inplace=True)
data = data.loc[data.index.get_level_values('Date').year>=2012 ]

Index(['20100104', '20100105', '20100106', '20100107', '20100108', '20100111',
       '20100112', '20100113', '20100114', '20100115',
       ...
       '20240923', '20240924', '20240925', '20240926', '20240927', '20240930',
       '20241008', '20241009', '20241010', '20241011'],
      dtype='object', name='Date', length=3586)

In [15]:
import rqdatac 
rqdatac.init('license', 'my_license_key')
rqdatac.info()

import pandas as pd
import numpy as np
import optuna
import operator #加入新的feature

from pandas import read_parquet
import xgboost as xgb
import pickle


from rqfactor import *
# from rqfactor.extension import *
# from rqfactor.notebook import *
import rqdatac
#实例化引擎
engine=FactorAnalysisEngine()
#构建管道，对因子进行预处理
# engine.append(('winzorization-mad', Winzorization(method='mad')))
# engine.append(('neutralization', Neutralization(industry='citics_2019', style_factors=['size'])))
# engine.append(('normalization', Normalization()))

#构建管道，添加因子分析器
engine.append(('rank_ic_analysis', ICAnalysis(rank_ic=True, industry_classification='sws',max_decay=20)))
engine.append(('quantile', QuantileReturnAnalysis(quantile=10, benchmark=None)))
engine.append(('return',FactorReturnAnalysis()))



/home/cheam/anaconda3/envs/galaxy/lib/python3.11/site-packages/rqdatac/client.py:245: UserWarning: Your account will be expired after  7 days. Please call us at 0755-22676337 to upgrade or purchase or renew your contract.
  warnings.warn("Your account will be expired after  {} days. "


rqdatac version: 3.0.10.1
server address: rqdatad-pro.ricequant.com:16011
You are using license:
QULvO2AwMWdObEcsI5JyYn9huXDVheJCAXY5NYxJ9HwG6NXDrCW4Hr0_phsk5zs0JSfyaVaHvkBit6EXmTbqaC7P1zXV32ciLWQSck5o9tYaUeNZvAbXssyfZpdBHdoytBKcyEM0cteZ4936V9nNv1pE0Fe3dE27fOHkNzGyCQk=eovAhRhDaTq3agpOUvzFLEuo_VOJrtG3E1f5rTpPOuvEGEXoUiupWvENKt0saJSxYCiOUw8CFQATFo3Bchb2c2E26AgG5KB9uFrCTz1v03L1zzcVjxGuDI0uqdP82kErOWD5eJ0lKSXBjJU-MOjB9wgSgmLBm2PwsTVvC5nYkF8=


In [16]:
path = "/home/cheam/IP_alpha/xgb_2y/"
name = "xgb_2y_mse"
factor = pd.read_parquet(path + "results_xgb.parquet")[['mean']]
factor=factor.reset_index(drop=False)
factor['Symbol']=rqdatac.id_convert((factor['Symbol']).tolist())
# #df_reset
factor=factor.set_index(['Date','Symbol'])
factor_pivot = pd.pivot_table(factor,index='Date', columns='Symbol', values='mean')
factor

mean
Date       Symbol               
2012-01-02 000001.XSHE -0.000159
           000002.XSHE -0.000183
           000004.XSHE -0.000045
           000005.XSHE -0.000044
           000006.XSHE -0.000117
...                          ...
2024-09-23 688799.XSHG  0.000838
           688800.XSHG  0.001439
           688819.XSHG -0.001562
           688981.XSHG  0.000079
           689009.XSHG  0.000419

[2360004 rows x 1 columns]

In [17]:
label=data.copy()
label = label.unstack().pct_change(1).shift(-1).stack()
label=label.reset_index(drop=False)
label['Symbol']=rqdatac.id_convert((label['Symbol']).tolist())  
label=label.set_index(['Date','Symbol'])
label_use = label.reindex(factor.index)
#df_pivot.index = pd.DatetimeIndex(factor.index.levels[0])
label_pivot= pd. pivot_table(label_use, index='Date', columns='Symbol', values='AdjVWAP')

In [ ]:
result = engine.analysis(factor_pivot,label_pivot , ascending=False, periods=[1,4], keep_preprocess_result=True)

In [19]:
ic_value = result['rank_ic_analysis'].summary()
ic_value.rename(columns=lambda x: name + '_' + x , inplace=True)

In [20]:
with open(path + 'analysis_result.pkl', 'wb') as f:
    pickle.dump(result, f)

In [21]:
result['rank_ic_analysis'].summary()

,xgb_2y_mse_P_1,xgb_2y_mse_P_4
mean,6.195201e-02,8.490513e-02
std,1.069464e-01,1.055162e-01
positive,4.810000e+02,5.290000e+02
negative,1.760000e+02,1.280000e+02
significance,7.360483e-01,7.948718e-01
sig_positive,5.768645e-01,6.727549e-01
sig_negative,1.643836e-01,1.263318e-01
t_stat,1.484814e+01,2.062517e+01
p_value,3.297956e-43,3.105514e-73
skew,-2.447546e-01,-2.311743e-01


In [25]:
import pickle
import pandas as pd
list = ["/home/cheam/IP_alpha/xgb_rolling_mse", "/home/cheam/IP_alpha/xgb_2y", 
    "/home/cheam/IP_alpha/xgb_ic_rolling", "/home/cheam/IP_alpha/xgb_ic_2y"]
all = []
for path in list:
    with open(path + '/analysis_result.pkl', 'rb') as f:
        tmp = pickle.load(f)
        tmp1 = tmp['rank_ic_analysis'].summary().copy()
        all.append(tmp1)
all = pd.concat(all, axis=1, join="outer", sort=True)
all

,xgb_rolling_mse_P_1,xgb_rolling_mse_P_4,xgb_2y_mse_P_1,xgb_2y_mse_P_4,xgb_rolling_ic_P_1,xgb_rolling_ic_P_4,xgb_2y_ic_P_1,xgb_2y_ic_P_4
ir,5.914934e-01,8.630556e-01,5.792810e-01,8.046646e-01,5.274276e-01,7.535726e-01,4.345612e-01,6.340216e-01
kurtosis,2.554602e-01,6.692715e-02,8.573073e-02,1.116335e-01,8.453200e-01,3.449552e-01,4.461867e-01,2.623936e-01
mean,6.369107e-02,8.959708e-02,6.195201e-02,8.490513e-02,4.949839e-02,6.911793e-02,4.124564e-02,5.903189e-02
negative,1.840000e+02,1.260000e+02,1.760000e+02,1.280000e+02,1.880000e+02,1.490000e+02,2.110000e+02,1.690000e+02
p_value,9.839353e-45,1.934430e-81,3.297956e-43,3.105514e-73,6.492886e-37,3.732595e-66,1.635911e-26,3.705247e-50
positive,4.730000e+02,5.310000e+02,4.810000e+02,5.290000e+02,4.690000e+02,5.080000e+02,4.460000e+02,4.880000e+02
sig_negative,1.598174e-01,7.914764e-02,1.643836e-01,1.263318e-01,1.476408e-01,1.065449e-01,1.765601e-01,1.263318e-01
sig_positive,5.875190e-01,6.560122e-01,5.768645e-01,6.727549e-01,5.372907e-01,6.194825e-01,5.007610e-01,5.753425e-01
significance,7.420814e-01,7.315234e-01,7.360483e-01,7.948718e-01,6.802413e-01,7.224736e-01,6.726998e-01,6.983409e-01
skew,-2.111259e-01,-1.223232e-02,-2.447546e-01,-2.311743e-01,-2.883674e-01,-9.631175e-02,-3.207132e-01,-2.504001e-01
